In [ ]:
!pip install opencv-python matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Đọc ảnh và chuyển sang ảnh grayscale
image = cv2.imread('lena256.jpg', cv2.IMREAD_GRAYSCALE)

# Sử dụng Canny để phát hiện viền và tạo nhãn (label)
edgesCanny = cv2.Canny(image, threshold1=100, threshold2=200)
edgesCannyTest = cv2.Canny(image, threshold1=150, threshold2=200)

# Hiển thị ảnh gốc và viền
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.title('Ảnh gốc')
plt.imshow(image, cmap='gray')
plt.subplot(1, 3, 2)
plt.title('Viền được phát hiện')
plt.imshow(edgesCanny, cmap='gray')
plt.subplot(1, 3, 3)
plt.title('Viền được phát hiện')
plt.imshow(edgesCannyTest, cmap='gray')
plt.show()


In [ ]:
from tensorflow.keras import layers, models

# Xây dựng mô hình CNN
model = models.Sequential()

# Lớp tích chập đầu tiên
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(None, None, 1), padding='same'))  # Chấp nhận input với độ phân giải bất kỳ
# model.add(layers.BatchNormalization())

# Lớp tích chập thứ hai
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# Lớp tích chập thứ ba
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# Lớp kết thúc với 1 kênh đầu ra tương ứng với các viền (edges)
model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

# Tổng hợp mô hình
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
import os

# Đường dẫn đến thư mục chứa ảnh
image_dir = './train2017/'

# Tạo danh sách các đường dẫn đến ảnh
list_of_image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')]

# # Kiểm tra danh sách đường dẫn
# 
print(len(list_of_image_paths))


In [ ]:
# Hàm tiền xử lý ảnh
def preprocess_image(image):
    image = image / 255.0  # Chuẩn hóa pixel về khoảng [0, 1]
    image = np.expand_dims(image, axis=-1)  # Thêm kênh (vì grayscale)
    return image

In [ ]:
# Chuẩn bị tập dữ liệu huấn luyện
X_train = []
y_train = []

i = 0
for img_path in list_of_image_paths:
    if i == 1000:
        break

    # if i < 1000:
    #     i+=1
    #     continue

    # Đọc ảnh và chuyển sang ảnh grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 150, 200)

    # Tiền xử lý ảnh và nhãn
    img_processed = preprocess_image(img)
    edges_processed = preprocess_image(edges)

    X_train.append(img_processed)
    y_train.append(edges_processed)

    i += 1

# Chuyển thành numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)

print("Kích thước dữ liệu:", X_train.shape, y_train.shape)


In [1]:
from keras import models
model = models.load_model('6000-50epochs.hdf5')

In [ ]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

# Vẽ biểu đồ loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
models.save_model(model, '370k-2000-epochs20-batchsize16.hdf5')

In [ ]:
img_ptgr = cv2.imread('phtgrphr256.jpg', cv2.IMREAD_GRAYSCALE)

img_ptgr_canny = cv2.Canny(img_ptgr, threshold1=100, threshold2=200)
img_ptgr_processed = preprocess_image(img_ptgr)
img_ptgr_predicted = model.predict(np.expand_dims(img_ptgr_processed, axis=0))[0]

plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.imshow(img_ptgr, cmap='gray')
# plt.subplot(1, 4, 2)
# plt.imshow(traffic_img_gray, cmap='gray')
plt.subplot(1, 3, 2)
plt.imshow(img_ptgr_canny, cmap='gray')
plt.subplot(1, 3, 3)
plt.imshow(img_ptgr_predicted, cmap='gray')

In [ ]:
betuan_img = cv2.imread('betuan.jpg', cv2.IMREAD_GRAYSCALE)
betuan_canny = cv2.Canny(betuan_img, threshold1=150, threshold2=200)
betuan_processed = preprocess_image(betuan_img)
betuan_predicted = model.predict(np.expand_dims(betuan_processed, axis=0))[0]

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.imshow(betuan_predicted, cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(betuan_canny, cmap='gray')

In [ ]:
gmc_img = cv2.imread('giac_mo_chieu.jpg', cv2.IMREAD_GRAYSCALE)
gmc_processed = preprocess_image(gmc_img)
gmc_predicted = model.predict(np.expand_dims(gmc_processed, axis=0))[0]

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(gmc_predicted, cmap='gray')

In [ ]:
# Dự đoán viền trên một ảnh mới
test_image = cv2.imread('lena256.jpg', cv2.IMREAD_GRAYSCALE)
test_image_processed = preprocess_image(test_image)
predicted_edges = model.predict(np.expand_dims(test_image_processed, axis=0))[0]

In [ ]:
# Hiển thị ảnh gốc và viền
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.title('Ảnh gốc')
plt.imshow(test_image, cmap='gray')
plt.subplot(1, 3, 2)
plt.title('Viền được tách qua Canny')
plt.imshow(edgesCannyTest, cmap='gray')
plt.subplot(1, 3, 3)
plt.title('Viền được dự đoán')
plt.imshow(predicted_edges.squeeze(), cmap='gray')
plt.show()